In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm

In [2]:
url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(url, headers=headers)
data = response.json()
with open('wb_catalogs_data.json', 'w', encoding='UTF-8') as file:
    json.dump(data, file, indent=2, ensure_ascii=False)
    print(f'Данные сохранены в wb_catalogs_data.json')

Данные сохранены в wb_catalogs_data.json


In [12]:
def get_catalogs_wb(target):
    """получение каталога вб"""
    url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers).json()

    with open('wb_catalogs_data.json', 'w', encoding='UTF-8') as file:
        json.dump(response, file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в wb_catalogs_data_sample.json')
    data_list = []
    for d in response:
        try:
            for child in d['childs']:
                if target == child['url']:
                    data_list.append({
                        'category_name': child['name'],
                        'category_url': child['url'],
                        'shard': child['shard'],
                        'query': child['query']})
                else:
                    for sub_child in child['childs']:
                        data_list.append({
                            'category_name': sub_child['name'],
                            'category_url': sub_child['url'],
                            'shard': sub_child['shard'],
                            'query': sub_child['query']})
        except:
            # print(f'не имеет дочерних каталогов *{d["name"]}*')
            continue

    return data_list


def search_category_in_catalog(target, catalog_list):
    """пишем проверку пользовательской ссылки на наличии в каталоге"""
    try:
        for catalog in catalog_list:
            if catalog['category_url'] == target:
                print(f'найдено совпадение: {catalog["category_name"]}')
                name_category = catalog['category_name']
                shard = catalog['shard']
                query = catalog['query']
                return name_category, shard, query
            else:
                # print('нет совпадения')
                pass
    except:
        print('Данный раздел не найден!')


def get_data_from_json(json_file):
    """извлекаем из json интересующие нас данные"""
    data_list = []
    for data in json_file['data']['products']:
        try:
            price = int(data["priceU"] / 100)
        except:
            price = 0
        data_list.append({
            'Наименование': data['name'],
            'id': data['id'],
            'Скидка': data['sale'],
            'Цена': price,
            'Цена со скидкой': int(data["salePriceU"] / 100),
            'Бренд': data['brand'],
            'id бренда': int(data['brandId']),
            'feedbacks': data['feedbacks'],
            'rating': data['rating'],
            'Ссылка': f'https://www.wildberries.ru/catalog/{data["id"]}/detail.aspx?targetUrl=BP'
        })
    return data_list


def get_content(shard, query, low_price=None, top_price=None):
    # вставляем ценовые рамки для уменьшения выдачи, вилбериес отдает только 100 страниц
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    data_list = []
    for page in tqdm(range(1, 101)):

        # url = f'https://wbxcatalog-ru.wildberries.ru/{shard}' \
        #       f'/catalog?appType=1&curr=rub&dest=-1029256,-102269,-1278703,-1255563' \
        #       f'&{query}&lang=ru&locale=ru&sort=sale&page={page}' \
        #       f'&priceU={low_price * 100};{top_price * 100}'
        url = f'https://catalog.wb.ru/catalog/{shard}/catalog?appType=1&curr=rub&dest=-1075831,-77677,-398551,12358499' \
              f'&locale=ru&page={page}&priceU={low_price * 100};{top_price * 100}' \
              f'&reg=0&regions=64,83,4,38,80,33,70,82,86,30,69,1,48,22,66,31,40&sort=popular&spp=0&{query}'
        data = requests.get(url, headers=headers).json()
        if len(get_data_from_json(data)) > 0:
            data_list.extend(get_data_from_json(data))
        else:
            print(f'Сбор данных завершен.')
            break
    return data_list


def save_excel(data, filename): 
    """сохранение результата в excel файл"""
    df = pd.DataFrame(data)
    writer = pd.ExcelWriter(f'{filename}.xlsx')
    df.to_excel(writer, 'data')
    writer.save()
    print(f'Все сохранено в {filename}.xlsx')


def parser(url, low_price, top_price):
    # получаем список каталогов
    target = url.split('https://www.wildberries.ru')[-1]
    catalog_list = get_catalogs_wb(target)
    try:
        # поиск введенной категории в общем каталоге
        name_category, shard, query = search_category_in_catalog(target, catalog_list=catalog_list)
        # сбор данных в найденном каталоге
        data_list = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        # сохранение найденных данных
        save_excel(data_list, f'{name_category}_from_{low_price}_to_{top_price}')
    except TypeError:
        print('Ошибка! Возможно не верно указан раздел. Удалите все доп фильтры с ссылки')
    except PermissionError:
        print('Ошибка! Вы забыли закрыть созданный ранее excel файл. Закройте и повторите попытку')


if __name__ == '__main__':
    """ссылку на каталог или подкаталог, указывать без фильтров (без ценовых, сортировки и тд.)"""
    # url = input('Введите ссылку на категорию для сбора: ')
    # low_price = int(input('Введите минимальную сумму товара: '))
    # top_price = int(input('Введите максимульную сумму товара: '))

    """данные для теста. собираем товар с раздела велосипеды в ценовой категории от 0 до 300 рубдей"""
    url = 'https://www.wildberries.ru/catalog/sport/vidy-sporta/komandnye-vidy-sporta'
    
 
    low_price = 0
    top_price = 300

    parser(url, low_price, top_price)

Данные сохранены в wb_catalogs_data_sample.json
найдено совпадение: Командные виды спорта


 59%|███████████████████████████████████████████████▊                                 | 59/100 [00:29<00:20,  2.03it/s]

Сбор данных завершен.


Все сохранено в Командные виды спорта_from_0_to_300.xlsx


In [26]:
df = pd.read_excel('Командные виды спорта_from_0_to_300.xlsx')
# Вайлдберрис ограничивает выдачу 100 страницами, на каждой по 100 карточек, возможный максимум для парсинга - 
# 10 000 карточек, поэтому ограничиваем поиск по ценовой категории, пусть будут самые дешевые товары - 
# до 300 рублей

In [19]:
df

,Unnamed: 0,Наименование,id,Скидка,Цена,Цена со скидкой,Бренд,id бренда,feedbacks,rating,Ссылка
0,0,Носки мужские набор белые черные короткие пода...,12246409,60,879,351,ECOBRAND,71300,16648,5,https://www.wildberries.ru/catalog/12246409/de...
1,1,Носки мужские набор высокие белые черные длинн...,12246403,59,919,376,ECOBRAND,71300,20397,5,https://www.wildberries.ru/catalog/12246403/de...
2,2,Носки мужские набор короткие спортивные серые ...,12246407,63,919,340,ECOBRAND,71300,16648,5,https://www.wildberries.ru/catalog/12246407/de...
3,3,Фитнес-резинки эспандер набор 5 шт,19197061,75,920,230,Резинки для фитнеса PWR!,938049,19902,5,https://www.wildberries.ru/catalog/19197061/de...
4,4,Носки мужские набор высокие белые черные длинн...,12246401,62,919,349,ECOBRAND,71300,20397,5,https://www.wildberries.ru/catalog/12246401/de...
...,...,...,...,...,...,...,...,...,...,...,...
5890,5890,Игла для мяча иголка насоса,160971873,65,500,175,игла иголки,310752948,0,0,https://www.wildberries.ru/catalog/160971873/d...
5891,5891,"Игла для мяча, футбольного, баскетбольного 10шт",161223350,0,199,199,А-Я,530250,0,0,https://www.wildberries.ru/catalog/161223350/d...
5892,5892,Носки женские низкие с принтом короткие мужски...,28620379,38,500,310,MOGZY,94646,194,5,https://www.wildberries.ru/catalog/28620379/de...
5893,5893,Носки женские низкие с принтом короткие мужски...,28620732,34,500,330,MOGZY,94646,223,5,https://www.wildberries.ru/catalog/28620732/de...


In [25]:
df['Наименование'].value_counts()
# До 300 рублей в разделе спортивных товаров для командных видов спорта можно в основном найти носки и нижнее белье
# Наименование разное, суть отдна

Носки                                                          245
Носки женские высокие с принтом длинные мужские спортивные      71
Гетры                                                           59
Гетры Спортивные                                                58
Носки женские высокие с принтом теплые плотные шерстяные        54
                                                              ... 
Шорты для мальчика для девочки спортивные                        1
Гольфы женские теплые белые высокие вязаные плотные высокие      1
Шайба хоккейная, хоккей, взрослая                                1
Носки женские короткие набор спорт повседневные в подарок        1
желтая футболка для малышей                                      1
Name: Наименование, Length: 2065, dtype: int64

In [38]:
!pip install nltk


In [43]:
!pip install collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [44]:
# Надо попробовать лемматизировать наименование
#
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4') # сработало?
from nltk.stem import WordNetLemmatizer
from collections import Counter


In [53]:
# Инициализируем лемматизатор
lemmatizer = WordNetLemmatizer()


In [54]:
# Создаем функцию для лемматизации текста
def lemmatize_text(text):
    words = text.lower().split()
    lemmas = [lemmatizer.lemmatize(word) for word in words]
    return lemmas


In [55]:
# Применяем функцию lemmatize_text ко всем значениям столбца 'Наименование'
lemmatized_words = df['Наименование'].apply(lemmatize_text).sum()


In [56]:
# Посчитаем наиболее часто встречающиеся леммы с помощью Counter
most_common_lemmas = Counter(lemmatized_words).most_common()


In [57]:
# Посчитаем наиболее часто встречающиеся леммы с помощью Counter
most_common_lemmas

[('носки', 2098),
 ('для', 1735),
 ('с', 1113),
 ('высокие', 1056),
 ('спортивные', 878),
 ('гетры', 804),
 ('мужские', 720),
 ('принтом', 703),
 ('женские', 683),
 ('набор', 601),
 ('длинные', 597),
 ('гольфы', 509),
 ('футбольные', 453),
 ('и', 403),
 ('на', 333),
 ('пары', 312),
 ('1', 307),
 ('белые', 306),
 ('спортивный', 293),
 ('хлопок', 290),
 ('кинезио', 277),
 ('короткие', 269),
 ('тела', 268),
 ('лица', 262),
 ('тейп', 257),
 ('в', 255),
 ('флаг', 247),
 ('пара', 242),
 ('детские', 240),
 ('2', 230),
 ('мяч', 212),
 ('3', 201),
 ('наколенники', 194),
 ('спортивная', 194),
 ('5', 191),
 ('шт', 190),
 ('милые', 187),
 ('свисток', 183),
 ('прикольные', 180),
 ('черные', 163),
 ('рисунком', 163),
 ('пар', 160),
 ('парные', 157),
 ('детская', 154),
 ('шайба', 154),
 ('тейпы', 148),
 ('подарок', 147),
 ('бандаж', 145),
 ('рубчик', 142),
 ('мальчика', 137),
 ('см', 135),
 ('россии', 135),
 ('теплые', 127),
 ('+', 126),
 ('чехлы', 122),
 ('коньков', 116),
 ('танцев', 115),
 ('бутылк

In [63]:
most_common_lemmas[0:30]

[('носки', 2098),
 ('для', 1735),
 ('с', 1113),
 ('высокие', 1056),
 ('спортивные', 878),
 ('гетры', 804),
 ('мужские', 720),
 ('принтом', 703),
 ('женские', 683),
 ('набор', 601),
 ('длинные', 597),
 ('гольфы', 509),
 ('футбольные', 453),
 ('и', 403),
 ('на', 333),
 ('пары', 312),
 ('1', 307),
 ('белые', 306),
 ('спортивный', 293),
 ('хлопок', 290),
 ('кинезио', 277),
 ('короткие', 269),
 ('тела', 268),
 ('лица', 262),
 ('тейп', 257),
 ('в', 255),
 ('флаг', 247),
 ('пара', 242),
 ('детские', 240),
 ('2', 230)]

In [81]:
# Самые широко представленные товары в этой ценовой категории - носки, гетры, гольфы, тейпы, флаги